In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm
import sys
import os

sys.path.append(os.path.abspath('/Users/engchongyock/Desktop/XCPatchTST'))
from config import get_config
from utils.helper import create_crosschannel_patchtst
from data.data_loader import create_dataloader
from utils.train import train

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [6]:
torch.manual_seed(42)
device = 'cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu')
print(f'Device: {device}')

Device: mps


In [9]:
df = pd.read_csv('/Users/engchongyock/Desktop/XCPatchTST/data/dataset/dataset_2025-03-16.csv')
df.sort_values(by='Date', inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop(columns=['Date'], inplace=True)
df

,AAPL_Adjusted Close,AAPL_EMA,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Effective_Fed_Rate,AAPL_MACD,AAPL_MACD_Signal,AAPL_MACD_Hist,AAPL_RSI,AAPL_SMA
0,2.575472,2.3837,1.20340,1.21420,1.20240,1.21350,4.30,0.1035,0.0756,0.0278,81.8085,2.3327
1,2.548998,2.4192,1.21240,1.21400,1.20480,1.21050,4.32,0.1131,0.0831,0.0300,82.7364,2.3772
2,2.482196,2.4339,1.21010,1.21580,1.20680,1.21130,4.24,0.1131,0.0891,0.0240,68.4255,2.4096
3,2.378230,2.4359,1.21090,1.21240,1.20510,1.20960,4.23,0.1074,0.0928,0.0146,60.3272,2.4280
4,2.289582,2.4267,1.20940,1.21390,1.20380,1.21360,4.24,0.0970,0.0936,0.0034,52.8462,2.4414
...,...,...,...,...,...,...,...,...,...,...,...,...
4815,239.070000,238.0299,1.07820,1.08888,1.07768,1.08300,4.33,0.1874,1.0961,-0.9086,43.6418,239.8070
4816,227.480000,237.5772,1.08392,1.08747,1.08048,1.08340,4.33,-0.0411,0.8687,-0.9097,44.6209,238.8685
4817,220.840000,235.0732,1.08345,1.09472,1.08305,1.09110,4.33,-1.1558,0.4638,-1.6196,29.3418,236.4490
4818,216.980000,232.3581,1.09111,1.09314,1.08754,1.08857,4.33,-2.3083,-0.0907,-2.2177,26.2255,234.0300


In [11]:
# Initialize a dictionary to store NMAE for each column
nmae_dict = {}

# Initialize a variable to accumulate the total NMAE
total_nmae = 0

# Iterate over each column to calculate NMAE
for column in df.columns:
    y_true = df[column].values

    # Generate naive predictions (shift the actual values by 1)
    naive_predictions = np.roll(y_true, 1)
    naive_predictions[0] = y_true[0]  # Set the first prediction to be the first actual value

    # Calculate MAE
    mae = np.mean(np.abs(y_true - naive_predictions))

    # Calculate the range of the true labels (target variable)
    range_labels = np.max(y_true) - np.min(y_true)

    # Calculate Normalized MAE (NMAE)
    nmae = mae / range_labels if range_labels != 0 else 0  # Avoid division by zero if the range is 0

    # Store the NMAE for this column
    nmae_dict[column] = nmae

    # Accumulate the NMAE to calculate the average later
    total_nmae += nmae

# Calculate the average NMAE
avg_nmae = total_nmae / len(df.columns)

# Print the results for each column and the average NMAE
for column, nmae in nmae_dict.items():
    print(f"NMAE for {column}: {nmae:.4f}")

print(f"\nAverage NMAE: {avg_nmae:.4f}")

NMAE for AAPL_Adjusted Close: 0.0028
NMAE for AAPL_EMA: 0.0010
NMAE for AAPL_Open: 0.0081
NMAE for AAPL_High: 0.0070
NMAE for AAPL_Low: 0.0068
NMAE for AAPL_Close: 0.0081
NMAE for AAPL_Effective_Fed_Rate: 0.0032
NMAE for AAPL_MACD: 0.0060
NMAE for AAPL_MACD_Signal: 0.0045
NMAE for AAPL_MACD_Hist: 0.0114
NMAE for AAPL_RSI: 0.0611
NMAE for AAPL_SMA: 0.0010

Average NMAE: 0.0101


In [ ]:
# tickers = ['AAPL', 'MSFT', 'META', 'GOOGL', 'AMD', 'AVGO', 'MU', 'NVDA']